# Улучшение сходимости нейросетей и борьба с переобучением.

## Сигмоида затухает и теоретически и практически

In [ ]:
(распределение весов?)

Глубокие нейронные сети склонны к двум эффектам: 

1. Переобучение
2. Затухание/взрыв градиента и как следствие паралич сети

## Инициализация весов

Хотим задать изначальные веса в нейросети. 
Как это можно сделать? 

Делать инициализацию 0 - плохая идея. В этом случае нейросеть получим, что градиент по всем таким весам будет одинаков - и их обновление, следовательно, тоже. Ничего не выучим. 

Можно инициализировать веса нормальным шумом с матожиданием 0 и маленькой дисперсией. Маленькая дисперсия нужна затем, чтобы не получить огромные градиенты за большие изначальные ошибки в предсказании. 

Но почему бы не взять другое распределение? Может, веса, инициализированные экспоненциальным распределениям дают какие-нибудь классные гарантии? Или какой-нибудь другой, не приходящий сразу в голову способ инициализации. Давайте это обсудим. 


### Инициализация Ксавьера (Xavier, Glorot)

Рассмотрим вкачестве активации нечетную функцию с единичной производной в нуле .



Например, нам подойдет гиперболический тангенс (tanh)

<img src ="src/img/tanh.png" width="300">

Мы хотим начать из линейного региона этой функции, чтобы избежать
затухающих градиентов

Как у нас зависят активации на текущем слое от активаций на предыдущем?

$$z^{i+1} = f(z^iW^i)$$

Тогда, так как мы вначале хотим находиться в районе линейности нашей функции, то

$$z^{i+1} \approx z^i W^i$$

Ответить на вопрос, а как будут связаны дисперсии этих активаций сложнее 

Сначала распишем:
1. чему равна дисперсия суммы двух независимых величин

$$D(\eta + \gamma) = D\eta + D\gamma$$

2. чему равна дисперсия произведений двух независимых величин


$$D\eta\gamma = E(\eta\gamma)^2 - (E\eta\gamma)^2 = E\eta^2E\gamma^2 - (E\eta)^2(E\gamma)^2$$ 

Далее распишем для одного веса текущего слоя 

$$z^{i+1}_{k} = \sum_t z^i_t w_{kt}$$

$$D(z^{i+1}_{k}) = D(\sum_t z^i_t w_{kt}) = \sum_t D(z^i_t w_{kt})$$

Предполагая, что дисперсии весов и активаций одинаковые (а нам бы так хотелось)

$$D(z^{i+1}_{k}) = n D(z^i_0 w_{k0})$$

Далее применяем нашу формулу и получаем:

$$D(z^{i+1}_{k}) = n [E(z^i_0)^2E(w_{k0})^2 - (Ez^i_0)^2(Ew_{k0})^2]$$

Хотим, чтобы что матожидание наших активаций 0, и мы можем этого добиться, делая  матожидание весов равным 0. 

$$D(z^{i+1}_{k}) =   n E(z^i_0)^2E(w_{k0})^2 $$

Заметим, что так как матожидание активация и весов равны 0, то матожидания их квадратов равны дисперсии активаций и весов соответственно

$$Dz = E(z^{i+1}_{k})^2 - (Ez^{i+1}_{k})^2 = E(z^{i+1}_{k})^2$$

$$D(z^{i+1}_{k}) = n Dz^i_0Dw_{k0}$$




Отсюда можно вывести формулу для зависимости активаций любого слоя от весов предуыдущих слоев и дисперсии исходных данных

$$Dz^i = Dx \prod_{p=0}^{i-1}n_pDW^p $$

Где n_p - размерность выхода слоя p-го слоя

Аналогично можно вывести формулу для градиентов по активациям


$$D(\dfrac {\delta L} {\delta z^i}) = D(\dfrac {\delta L} {\delta z^d} ) \prod_{p=i}^{d}n_{p+1}DW^p $$



Для хорошей инициализации нам бы хотелось, чтобы дисперсии активаций на всех слоях были одинаковые. 
А еще хотелось бы, чтобы дисперсии градиентов тоже были одинаковые. 

Тогда у нас не происходит резких скачков в распределении активаций, а градиент не затухает и не взрывается 


$$Dz^i = Dz^j$$
$$D\dfrac {\delta L} {\delta z^i} = D\dfrac {\delta L} {\delta z^j}$$


Учитывая предыдущее, это эквивалентно тому, что мы требуем

$$n_iDW^i = 1$$
$$n_{i+1}DW^i = 1$$

Одновременно так сделать не получится $$n_i \ne n_{i+1}$$

Потому делаем компромисс - среднее гармонческое решений первого и второго уравнения

$$DW^i = \dfrac 2 {n_i + n_{i+1}}$$

Надо выбрать распределение.

$$ EW^i = 0 $$

$$DW^i = \dfrac 2 {n_i + n_{i+1}}$$

Можно было бы взять нормальное распределение с такими параметрами.

$$W_i \sim N(0, sd=\sqrt{\dfrac 2 {n_i + n_{i+1}}}) $$

А можно равномерное:

$$D(U[a, b]) = \dfrac 1 12 (b -a)^2$$

$$W_i \sim U[-\dfrac {\sqrt{6}} {n_i + n_{i + 1}}, \dfrac {\sqrt{6}} {n_i + n_{i + 1}} ]$$



### He-инициализация

Но многие люди используют ReLU. Она в 0 не имеет производной, потому применять инициализацию, указанную выше, проблематично. 

Тогда получатся похожие условия 

$$Dz^i = Dx \prod_{p=0}^{i-1}\dfrac 1 2 n_pDW^p $$

$$D(\dfrac {\delta L} {\delta z^i}) = D(\dfrac {\delta L} {\delta z^d} ) \prod_{p=i}^{d}\dfrac 1 2 n_{p+1}DW^p $$


И для них решения будут тоже похожими:

$$\frac 2 {n_k}$$

и

$$\frac 2 {n_{k+1}}$$

Можно опять взять среднее гармоническое. Но на практике, особенно в случае сверточных нейронных сетей, просто берут либо $ \frac 2 {n_i}$ либо $\frac 2 {n_i + 1}$

$$W^i \sim N(0, sd=\sqrt{\frac 2 n_i})$$

$$W^i \sim N(0, sd=\sqrt{\frac 2 {n_i + 1}})$$

Опять же, можно использовать и равномерное распределение

## Важность

От правильной активации может зависеть очень многое:

1. Нейросеть может сойтись значительно быстрее

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/lecture-07.png" width="500">

2. В зависимости от выбранной активации сеть вообще может сойтись или не сойтись

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/lecture-08.png" width="500">

### Обобщение инициализаций Ксавьера и He-инициализации

Вообще говоря, коэффициенты в инициализациях (числитель в формуле для дисперсии), зависит от конкретной выбранной функции активации.
В pytorch есть функции для вычисления этих коэффициентов
https://pytorch.org/docs/stable/nn.init.html


### Ортогональная инициализация

Также иногда используется так называемая ортогональная инициализация весов. 
Для каждого слоя мы убеждаемся, что изначальная матрица весов является ортогональной. 

Выберем ортогональную матрицу весов 
$$W: WW^T = 1$$. 

Тогда:
1.  норма активации сохраняется (опять же, активации между слоями остаются в одном масштабе)
$$||s_{i+1}|| = ||W_{i}s_i|| = ||s_i||$$

2.  все нейроны делают «разные» преобразования
$$ ⟨W_i, W_j⟩ = 0~i \ne j$$
$$ ⟨W_i, W_j⟩ = 1~i = j$$


Иногда такая инициализация обеспечивает значительно лучшую сходимость https://datascience.stackexchange.com/questions/64899/why-is-orthogonal-weights-initialization-so-important-for-ppo

## Регуляризация

Второй способ улучшения сходимости нейросетей и борьбы с переобучением - введение регуляризации. Ее можно вводить несколькими способами. 

### L1, L2 регуляризации

Самый простой способ мы уже разобрали - давайте просто добавим в лосс штраф к весам

$$Loss\_reg = loss + \lambda \cdot reg$$

$$ reg_{L1} = \lambda \sum |w_i| $$

$$ reg_{L2} = \lambda \sum w_i^2 $$

<img src="https://edunet.kea.su/repo/src/L0X_Encoders/img/losses.gif" alt="alttext" style="width: 500px;"/>

Иногда уже его хватает, чтобы решить все проблемы. Напомним, что L2 лосс приводит к большому числу маленьких ненулевых весов в сети. А L1 лосс - к маленькому числу ненулевых весов (разреженной нейросети)

### Dropout

Одним из распространненных именно в нейросетях методом регуляризации является Dropout.

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/lecture-12.png" width="500">

Состоит этот метод в следующем:

1. Во время обучения мы с вероятностью *p* зануляем выход нейронов слоя (например, *p* = 0.5)
2. Зануленные нейроны не участвуют в данном forward, и градиент потому к ним при backward не идет. 

3. Сила регуляризации определяется вероятностью p, чем она больше - тем сильнее регуляризация. 


### Мотивация Dropout

#### Борьба с коадаптацией 

Одной из  проблем при работе с глубокими сетями является совместная адаптация нейронов. В такой сети, если все весовые коэффициенты изучаются вместе, обычно некоторые соединения будут иметь больше возможностей прогнозирования, чем другие.

Часть нейронов делает основную работу - предсказывает, а остальные могут вообще не вносить никакого вклада в итоговое предсказание. Или же другая картина - один нейрон делает кривоватое предсказание, другие его правят и в итоге первый нейрон своей ошибки не исправляет. 

Это явление называется со-адаптацией. Этого нельзя было предотвратить с помощью традиционной регуляризации, такой как L1 и L2. А вот Dropout с этим хорошо борется

Отмена способности некоторых нейронов учиться во время тренировки на самом деле оказывается направлена на то, чтобы получить более обученные нейроны и сократить переобучение, гарантирует, что выучиваемые индивидуальными нейронами функции хорошо работают со случайно выбранными подмножествами функций, выученных другими нейронами. 
Таким образом,  Dropout приводит к появлению более требовательных нейронов, которые пытаются обойти сложные, индивидуальные особенности, которые склонны плохо обобщать, и сохраняют больше полезной информации самостоятельно. 

На следующем рисунке (извлеченном из статьи Dropout: A Simple Way to Prevent Neural Networks from Overfitting) мы находим сравнение признаков, изученных в наборе данных MNIST с одним автоэнкодером скрытого слоя, имеющим 256 выпрямленных линейных единиц без отсева (слева), и признаков, изученных той же структурой с использованием отсева в ее скрытом слое с p=0,5 (справа).

В то время как первый показывает неструктурированные, беспорядочные паттерны, которые невозможно интерпретировать, второй явно демонстрирует целенаправленное распределение веса, которое обнаруживает штрихи, края и пятна самостоятельно, нарушая их взаимозависимость с другими нейронами для выполнения этой работы.

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/lecture-13.png" width="400">

#### Dropout как регуляризация 

Фактически, Dropout штрафует слишком сложные, неустойчивые решения. Добавляя в нейросеть Dropout мы сообщаем ей о том, что решение, которое мы ожидаем, должно устойчиво к шуму

#### Dropout как ансамбль 

Можно рассматривать Dropout как ансамбль нейросетей со схожими параметрами, которые мы учим одновременно, вместо того, чтобы учить каждую в отдельности, а затем результат их предсказания усредняем, замораживая Dropout (http://mlg.eng.cam.ac.uk/yarin/blog_3d801aa532c1ce.html)

Фактически, возникает аналогия со случайным лесом - каждая из наших нейросетей легко выучивает выборку и переобучается - имеет низкий bias, но высокий variance. При этом за счет временного отключения активаций, каждая нейросеть видит не все объекты, а только часть. Усредняя все эти предсказания, мы уменьшаем variance. 

#### Confidence interval от Dropout
Можно используя нейросеть с дропаутом, получить доверительный интервал для вашего предсказания. Просто не "замораживаем" dropout-слои во время предсказания, а делаем предсказания с активными dropout. 

И делаем forward через такую нейросеть для одного объекта 1000 раз. 
Сделав это 1000 раз вы получаете распределение предсказаний, на основе которого можно делать confidence интервалы и как раз ловить те объекты, на которых нейросетей вообще не понимает, что ей делать и потому предсказывает метку или еще что-то с сильной дисперсией. 



<img src ="src/img/confidence_nn.png" width="400">


In [ ]:
ссылка

**Самая простая реализация для трёхслойной нейросети выглядит следующим образом:**

In [7]:
""" Vanilla Dropout: не рекомендуется использовать на практике """

p = 0.5 # вероятность оставить нейрон активным. Чем выше, тем меньше dropout

def train_step(X):

  """ X - данные """

  # прямой проход по трёхслойной нейросети

  H1 = np.maximum(0, np.dot(W1, X) + b1)

  U1 = np.random.rand(*H1.shape) < p # первый dropout

  H1 *= U1 # drop!

  H2 = np.maximum(0, np.dot(W2, H1) + b2)

  U2 = np.random.rand(*H2.shape) < p # второй dropout

  H2 *= U2 # drop!

  out = np.dot(W3, H2) + b3

  # обратный проход: вычисление градиентов... (не показано)

  # обновление параметров... (не показано)

def predict(X):

  # ансамблевый прямой проход

  H1 = np.maximum(0, np.dot(W1, X) + b1) * p # масштабируем активации!

  H2 = np.maximum(0, np.dot(W2, H1) + b2) * p # масштабируем активации!

  out = np.dot(W3, H2) + b3

Нежелательное свойство представленной выше схемы состоит в том, что мы должны масштабировать активации во время прогнозирования. Поскольку производительность тестирования критически важна, лучше всего использовать инвертированный dropout, который выполняет масштабирование во время обучения. Кроме того, если мы захотим убрать dropout из кода, функция прогнозирования останется без изменений.

In [8]:
""" 

Inverted Dropout: Рекомендуемая реализация

"""

p = 0.5 # вероятность оставить нейрон активным. Чем выше, тем меньше dropout

def train_step(X):

  # прямой проход по трёхслойной нейросети

  H1 = np.maximum(0, np.dot(W1, X) + b1)

  U1 = (np.random.rand(*H1.shape) < p) / p # первый dropout. Заметьте /p!

  H1 *= U1 # drop!

  H2 = np.maximum(0, np.dot(W2, H1) + b2)

  U2 = (np.random.rand(*H2.shape) < p) / p # второй dropout. Заметьте /p!

  H2 *= U2 # drop!

  out = np.dot(W3, H2) + b3

  # обратный проход: вычисление градиентов... (не показано)

  # обновление параметров... (не показано)

def predict(X):

  # ансамблевый прямой проход

  H1 = np.maximum(0, np.dot(W1, X) + b1) # масштабирование не нужно

  H2 = np.maximum(0, np.dot(W2, H1) + b2)

  out = np.dot(W3, H2) + b3


### Dropconnect

Будем занулять не нейроны, а веса. Фактически - для каждого батча обрубаем часть связей между нейронами

<img src ="src/img/dropconnect.png" width="700">

Drop Connect случайным образом отбрасывая веса, а не активации с вероятностью *p*.

DropConnect похож на Dropout, поскольку он вводит динамическую разреженность в модель , но отличается тем, что разреженность зависит от весов *W*, а не от выходных векторов слоя. Другими словами, полностью связанный слой с DropConnect становится разреженно связанным слоем, в котором соединения выбираются случайным образом на этапе обучения. 

В принципе, вариантов зануления чего-то в нейросетке можно предложить великое множество, в разных ситуациях будут работать разные ([в этом списке](https://paperswithcode.com/methods/category/regularization)  много Drop...)

### DropBlock

Например, можно убирать для каждого батча из нейросети случайные блоки из слоев. И это будет работать!

<img src ="src/img/dropblock.png" width="700">

## Нормализация

### Нормализация входных данных

Представим себе, что данные, которые мы подаем в нейросеть, распределены следующщим образом

<img src ="src/img/no_normalization.png" width="300">


Фактически нейросети работают со скалярными произведениями. В этом плане две вектора, изображенных на рисунке, не сильно отличаются. Так же и точки нашего датасета слабо разделимы. Чтобы с этим работать, нейросеть сначала должна подобрать удобное преобразование, а затем только сравнивать наши объекты. Понятно, это усложняют задачу. 

Для того, чтобы облегчить нейросети задачу, входные признаки часто нормируют:

$$x1' = \dfrac {x1 - \mu_{x1}} {\sigma_{x1}}$$
$$x2' = \dfrac {x2 - \mu_{x2}} {\sigma_{x2}}$$

<img src ="src/img/normalization.png" width="300">


 Такое преобразование действительно помогает нейросети 
 
 <img src ="src/img/search_space.png" width="300">

[altext](src/img/search_space.png)

### Covariate shift

Covariate shift - явление, когда признаки тренировочной выборки и тестовой по-разному распределены.

<img src ="src/img/covariate_shift.png" width="250">

В такой ситуации модель не в состоянии делать адекватные предсказания на тесте, так как не видела области пространства, в которой расположены тестовые объекты

<img src ="src/img/covariate_shift2.png" width="250">

### Internal covariate shift

Похожее явление может иметь место уже внутри нейросети

Пусть у нас iй слой переводит выдачу i-1 в новое пространство. 

<img src ="src/img/internal_covariate_shift.png" width="250">

В конце нейросеть делает предсказание, считается лосс, делается обратное распространение ошибки и обновляются веса. 

<img src ="src/img/internal_covariate_shift2.png" width="250">

После этого возникает нехорошая ситуация - распределение выходов i-1 слоя поменялось, а i-й слой изменял веса, думая, что распределение выходов не изменилось

<img src ="src/img/internal_covariate_shift3.png" width="250">

### Плохой вариант борьбы с этим

Давайте на каждом слое просто нормировать каждый признак, используя среднее и дисперсию по батчу

$$\tilde{x}_i^j = \dfrac {x_i^j - Ex^j} {\sqrt{Dx^j}}$$




Проблема в том, что таким образом мы можем попасть в область линейной составляющей нашей функции. Например, в случае сигмоиды 

<img src ="src/img/lin_sigmoid.png" width="350">


Получаем набор линейных слоев фактически без функций активации -> все вырождается в однослойную сеть. Не то что нам надо

### BatchNormalization

Нам надо дать нейросетке возможность перемещать распределение слоя из области 0 и самой подбирать дисперсию 

<img src ="src/img/nn_example.png" width="200">


$$\tilde{x}_i^j = \dfrac {x_i^j - Ex^j} {\sqrt{Dx^j}}$$



$$\hat{x_i^j} = \gamma^j\tilde{x}_i^j + \beta^j$$

Фактически, теперь нейросеть даже может отменить нормализацию, если считает ее ненужной

#### Скользящее среднее

Во время предсказания батча у нас уже нет - откуда брать матожидание и дисперсию? 

1. Можно посчитать по всей обучающей выборке - не всегда удобно, долго
2. Считать скользящее среднее во время обучения

$$E^j_{k+1} = (1 - \alpha)E^j_k + \alpha E^j_{batch}$$

Используем обычно $\alpha = 0.1$

Аналогично считаем дисперсию 

#### Защита от нулей в знаменателе

Чтобы у нас не мог возникнуть 0 в знаменателе, добавляем маленькое чиссло - $\epsilon$. Например, равное 1e-5


$$\tilde{x}_i^j = \dfrac {x_i^j - Ex^j} {\sqrt{Dx^j + \epsilon}}$$

$$\hat{x_i^j} = \gamma^j\tilde{x}_i^j + \beta^j$$

#### Линейный слои и конволюции

Слой конволюции можно свести к линейному слою с очень жесткими ограничениями на веса. Поэтому неудивительно, что BatchNorm можно применять и для линейных слоев и для конволюций. 

С конволюциями есть единственный нюанс - у нас "одним признаком" считается вся получаемая **feature map**. 

<img src ="src/img/feature_map.png" width="450">

И нормализация идет по всей такой feature map (по всему каналу) для всех объектов. 

#### Пример работы

Этот метод действительно работает. 
Видим, что нейросети с батчнормализацией:

1. Сходятся быстрее, чем нейросети без 
2. Могут работать с более высоким начальным learning rate, причем это позволяет достигать лучших результатов
3. BatchNorm позволяет глубокой нейросетке работать даже с функцией активации в виде сигмоиды. Без батчнорма такая сеть не обучилась бы вовсе. 

<img src ="src/img/batchnorm_work.png" width="500">


#### Градиент

Вычисление градиента batchnorm - интересное упражнение на понимание того, как работает backpropagation. В лекции мы это опускаем, можете ознакомиться самостоятельно

[Вывод градиентов для весов слоя BatchNorm](https://kevinzakka.github.io/2016/09/14/batch_normalization/)



<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/lecture-20.png" width="700">

### Batchnorm как регуляризация

Почему для нейросети с батчнормализацией можно использовать более высокие learning rate? 

 

Оказывается, батчнормализация делает неявную регуляризацию на веса

Допустим, мы решили увеличить веса в $a$ раз

Так как мы шкалируем, то домножение весов W на константу выходных значений слоя не меняет

$$BN((aW)u) = BN(Wu)$$

Градиент слоя по входу не меняется

$$\dfrac {\delta BN((aW)u)} {\delta u} = \dfrac {\delta BN(Wu)} {\delta u}$$

А градиент по весам уменьшается в a раз

$$\dfrac {\delta BN((aW)u)} {\delta aW} = \dfrac 1 a \dfrac {\delta BN(Wu)} {\delta W} $$

Таким образом нейросеть автоматически не дает большим весам расти

### Internal covariate shift?

Согласно некоторым исследованиям ([например](https://arxiv.org/abs/1805.11604)), успех BatchNormalization не заключается в исправлении covariate shift. 

BatchNormalization работает как-то иначе, улучшая гладкость пространства решений и облегчает поиск в нем минимума.

<img src ="src/img/batchnorm_smooth.png" width="700">

### Tips

Стоит помнить, что с батч-нормализацией:

* **Крайне важно** перемешивать батчи между эпохами. Единицей обучения параметров $\beta$ и $\gamma$ являются батчи. Если их не перемешивать, то из 6400 объектов в тренировочном датасете получим лишь 100 объектов для обучения $\beta$ и $\gamma$

* В слое, после которого поставили BatchNormalization, надо убрать смещения (параметр $\beta$ в BatchNormalization берет эту роль сам по себе)


* Другое расписание learning rate: бОльшее значение в начале обучения и быстрое уменьшение в процессе обучения

* Если используем BatchNormalization, то надо уменьшить силу Dropout и L2-регуляризации

* Чем меньше размер батча в обучении, тем хуже будет работать BatchNormalization

<img src ="src/img/batchnorm_batch.png" width="400">


### Другие <Вставьслово>Normalization

[Множество их](https://paperswithcode.com/methods/category/normalization)

<img src ="src/img/normalization_methods.png" width="700">



Чаще всего нормализации тестируют на изображениях в том числе или только на изображениях. Потому их графически объясняют при помощи следующей схемы:

<img src ="src/img/notation_3d_tensor_viz.png" width="350">

По одной оси - каналы (feature maps), по второй - объекты из батча, а по третье feature map, reshapeнутого в одномерный вектор 



В этой терминологии BatchNorm выглядит следующим образом

<img src ="src/img/batch_norm_3d.png" width="350">

#### Layer Norm

Нормализуем активации не вдоль объектов, а вдоль признаков, для каждого объекта индивидуально. Хорошо работает для рекуррентных нейронных сетей


<img src ="src/img/layer_norm_3d.png" width="350">


#### Instance Norm

При пакетной нормализации происходит усреднение параметров по всему пакету. Например, в случае задачи переноса стилей картин, это вносит много шума. При усреднении теряются индивидуальные характеристики объектов. Поэтому используется более тонкая нормализация — индивидуальная нормализация (англ. instance normalization) (по каналу, но только для данного объекта)

<img src ="src/img/instance_norm_3d.png" width="350">

#### GroupNorm

Нормализуем активации по группам каналов (**feature_map**), как и в BatchNorm, но только для данного объекта.

Эффективен в случае, когда у нас батчи маленького размера (именно его мы сравнивали с обычной батч-нормализацией выше)

<img src ="src/img/group_normalization_3d.png" width="350">



### Weights standartization 3d

Будем нормализовать **веса** каждого выходного слоя отдельно. 
Это в теории улучшает ландшафт, в котором мы ищем минимум функции потерь.
Обратите внимание, что здесь оси другие

<img src ="src/img/weight_standartization3d.png" width="350">

За счет замены BatchNorm в популярной архитектуре ResNet на GroupNorm и Weights standartization была получена нейросеть, которую легко можно было адаптировать к разным задачам 

<img src ="src/img/resnet_modified.gif" width="350">


### Ставить BatchNormalization до или после активации?

#### До

<img src ="src/img/bn_act.png" width="350">


* Рекомендуется авторами статьи, где предложили Batch Normalization
* Для сигмоиды, BN, поставленная после активации, не решает проблем сигмоиды



#### После

<img src ="src/img/act_bn.png" width="350">

* Аргументация авторов статьи не до конца обоснованна
* Обычно, сигмоиду не используют в современных нейронных сетях
* Для популярной ReLU, BN, поставленная до активации может приводить к “умирающей ReLU”, когда большая часть ее входов меньше 0 и потому для них градиент не проходит
* На многих задачах BN после функции активации работает лучше или не хуже поставленной до

<img src ="src/img/bn_relu.png" width="500">

[ссылка](https://github.com/ducha-aiki/caffenet-benchmark/blob/master/batchnorm.md)

### Ставить BatchNormalization до или после Dropout?...

#### До
<img src ="src/img/bn_before.png" width="350">

* Меньше влияние (covariate shift) Dropout на Batchnorm

#### После

<img src ="src/img/bn_after.png" width="350">

* Информация о зануленных активациях не просачивается через среднее и дисперсию батча

#### Ставить только что-то одно 

* Dropout может отрицательно влиять на качество нейросети с BatchNorm за счет разного поведения на train и test




#### Строго говоря

* Оптимальный порядок следования слоев зависит от задачи и архитектуры сети
* Возможно, стоит применять модифицированные версии BatchNorm

## Оптимизация весов нейросетей

Методов тоже много, расскажем о популярных ([неполный список](https://paperswithcode.com/methods/category/stochastic-optimization))

### SGD

### NAG (Nesterov momentum)


### Adaptive Learning Rate

### Adagrad

### RMSprop

### Adam

### Сравнение оптимизаторов 

У каждого из предложенных оптимизаторов есть минусы и плюсы 



#### Методы с инерцией сходятся к решению более плавно, но могут "перелетать"

<img src ="src/img/opt_smooth.gif" width="250">


#### Методы с адаптивным learning rate быстрее сходятся, более стабильны и меньше случайно блуждают

<img src ="src/img/adaptive_stability.gif" width="250">


####  Алгоритмы без адаптивного learning rate сложнее выбираются из локальных минимумом

<img src ="src/img/local_minima.gif" width="250">



#### Алгоритмы с инерцией осцилируют в седловых точках прежде чем найти верный путь

<img src ="src/img/saddle_point.gif" width="250">



## Режимы обучения

Нам не обязательно поддерживать один и тот же learning rate в течении всего обучения. Мы можем менять его по некоторым правилам. 



### Ранняя остановка

Можем использовать критерий ранней остановки - когда лосс на валидационной выборке не улучшается patience эпох, умножаем learning rate на factor

<img src ="src/img/early_stopping.png" width="350">

In [11]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), 
                            lr=0.1)
scheduler = ReduceLROnPlateau(optimizer, 
                              'min', 
                              factor=0.1,
                              patience = 5)

In [13]:
import torch

In [14]:
?torch.optim.lr_scheduler.LambdaLR

### Домножать learning rate на alpha каждую эпоху

In [ ]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

### Cyclical learning schedule





#### Подбираем границы learning rate



#### Запускаем

### Взаимодействие learning schedule и адаптивного изменения learning rate

И то, и другое меняет learning rate, learning scheduler - глобально, а адаптивные оптимизаторы - для каждого веса отдельно 

In [ ]:
сдвинуть батчнорм выше dropout

нормализация входов в начале? а батчнорм в конце? 